<a href="https://colab.research.google.com/github/tejasbana/Blog-Website/blob/main/SuperMind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download dataset
!gdown --id 1HRnvWy07E5RLfltjDIzR6pn9UbXqb9uB
!gdown --id 1AOzCq5u7pqXCnaQ_ghGqPgSNb7SF0LyK
!gdown --id 1gj8PH7BglgL4nrA9PdP204QZvvDhQVye
!pip install contractions

In [ ]:
!python -m spacy download en_core_web_lg

In [5]:
# import lib
from collections import Counter
from IPython.display import display
import logging
import os
from pprint import pprint
import re
import string
import contractions
import gensim
from gensim.models.phrases import Phrases, Phraser
from graphviz import Source
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.parse.stanford import StanfordParser, StanfordDependencyParser
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from spacy import displacy
from tqdm.notebook import tqdm
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')
nltk.download('treebank')

sns.set_theme(style="darkgrid")
pd.set_option("display.max_columns", 100)
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [7]:
# load conversations
conv_dataframe = pd.read_csv('./data.csv')
display(conv_dataframe.head())
print("Shape of dataframe : {}".format(conv_dataframe.shape))

,id,content
0,321712,Hey 👋 \n\nWe re using our bot:\n\nhttps://t.me...
1,321713,Good stuff \n\nI am surprised I took so long t...
2,321717,you are using a non-official one
3,321718,use the one that uniswap uses: https://thegrap...
4,321719,keep in mind this is a hot subgraph so it can ...


Shape of dataframe : (44136, 2)


In [8]:
# load term_abb file
term_abb_dataframe = pd.read_csv('./term_abb.csv')
display(term_abb_dataframe.head())
print("Shape of dataframe : {}".format(term_abb_dataframe.shape))

# extract labels
target_labels = term_abb_dataframe['terms'].values
print(target_labels)

,terms,abbreviations
0,Auroracoin,AUR
1,BitConnect (inactive),BCC
2,Bitcoin Cash,BCH
3,Bitcoin,BTCorXBT
4,Dash,DASH


Shape of dataframe : (78, 2)
['Auroracoin' 'BitConnect (inactive)' 'Bitcoin Cash' 'Bitcoin' 'Dash'
 'Dogecoin' 'EOS.IO' 'Ethereum Classic' 'Ether (also known as Ethereum)'
 'Gridcoin' 'Litecoin' 'Coinye (inactive)' 'Mazacoin' 'Nano' 'Neo'
 'Namecoin' 'NXT' 'PotCoin' 'Peercoin' 'Titcoin' 'USD Coin (stablecoin)'
 'Tether' 'Vertcoin' 'NEM' 'Stellar' 'Monero' 'Primecoin' 'Ripple' 'Verge'
 'Zcash' 'Bitcoin Improvement Proposal'
 'Automatic Teller Machine for Bitcoin' 'Enterprise Ethereum Alliance'
 'Ethereum Improvement Proposal' 'Fundamental Analysis'
 'Moving Average Convergence Divergence' 'Proof of Burn'
 'Proof of Developer' 'Technical Analysis or Trend Analysis'
 'Coordinated Universal Time' 'Year to Date' 'Address'
 'Application Programming Interface' 'Brute Force Attack'
 'Bitcoin address format (also known as bc1 addresses)'
 'Developers Conference' 'Multi-Signature' 'Secure Hash Acronym (256-bit)'
 'Worldwide Developers Conference' 'Anti-Money Laundering' 'All-Time High'
 'All-Tim

In [9]:
# Create custom_mapping for contractions
custom_mapping = dict(term_abb_dataframe.values)
custom_mapping

{'Auroracoin': 'AUR',
 'BitConnect (inactive)': 'BCC',
 'Bitcoin Cash': 'BCH',
 'Bitcoin': 'BTCorXBT',
 'Dash': 'DASH',
 'Dogecoin': 'DOGEorXDG',
 'EOS.IO': 'EOS',
 'Ethereum Classic': 'ETC',
 'Ether (also known as Ethereum)': 'ETH',
 'Gridcoin': 'GRC',
 'Litecoin': 'LTC',
 'Coinye (inactive)': 'KOIorCOYE',
 'Mazacoin': 'MZC',
 'Nano': 'Nano',
 'Neo': 'NEO',
 'Namecoin': 'NMC',
 'NXT': 'Nxt',
 'PotCoin': 'POT',
 'Peercoin': 'PPC',
 'Titcoin': 'TIT',
 'USD Coin (stablecoin)': 'USDC',
 'Tether': 'USDT',
 'Vertcoin': 'VTC',
 'NEM': 'XEM',
 'Stellar': 'XLM',
 'Monero': 'XMR',
 'Primecoin': 'XPM',
 'Ripple': 'XRP',
 'Verge': 'XVG',
 'Zcash': 'ZEC',
 'Bitcoin Improvement Proposal': 'BIP',
 'Automatic Teller Machine for Bitcoin': 'BTM',
 'Enterprise Ethereum Alliance': 'EEA',
 'Ethereum Improvement Proposal': 'EIP',
 'Fundamental Analysis': 'FA',
 'Moving Average Convergence Divergence': 'MACD',
 'Proof of Burn': 'PoB',
 'Proof of Developer': 'PoD',
 'Technical Analysis or Trend Analysis': 'T

In [10]:
term_def_dataframe = pd.read_csv('./term_def.csv')
display(term_def_dataframe.head())
print("Shape of dataframe : {}".format(term_def_dataframe.shape))

,terms,definition1,definition2
0,51% attack,A hypothetical situation where more than half ...,NaN
1,51% attack protection,A protection mechanism implemented by several ...,NaN
2,AFK,Away From Keyboard; used on social media platf...,NaN
3,Airdrop,An event where a blockchain project distribute...,NaN
4,Altcoin,Any cryptocurrency that is an alternative to B...,NaN


Shape of dataframe : (160, 3)


In [11]:
# Separating conversations
all_conversations = list(conv_dataframe['content'].astype(str))  
# Splitting training and testing set. 80/20 split
train_conv, test_conv = train_test_split(all_conversations, test_size=0.2, random_state=42)

print("Size of training dataset: ",len(train_conv))
print("Size of testing dataset: ", len(test_conv))

Size of training dataset:  35308
Size of testing dataset:  8828


In [42]:
# text pre-processing
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def process_sentence(conversation):
    # Remove emoji
    conversation = remove_emoji(conversation)

    # Exapnding contractions
    conversation = contractions.fix(conversation)
    for x in custom_mapping.keys():
        conversation = re.sub(custom_mapping[x], x, conversation)
    
    #remove stop words
    stopwords_english = stopwords.words('english')

    conversation = re.sub(r'\$\w*', '', conversation)
    # remove hyperlinks    
    conversation = re.sub(r'https?://[^\s\n\r]+', '', conversation)
    # remove hashtags
    # only removing the hash # sign from the word
    conversation = re.sub(r'#', '', conversation)
    # # tokenize conversation
    conversation_tokens =  nltk.tokenize.word_tokenize(conversation) 

    conversation_clean = []
    for word in conversation_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # conversation_clean.append(word)
            # stemmer = PorterStemmer()
            # word = stemmer.stem(word)  # stemming word
            conversation_clean.append(word)

    return conversation_clean

In [43]:
# test pre-processing on example conversation
example = all_conversations[0]

print('\033[94m')
print(example)
print('\033[92m')
print("After processing: \n", process_sentence(example))


Hey 👋 

We re using our bot:

https://t.me/lobster_watcher

And also filtering such recommendations to select only topics worth attention.

~5 people are in duty every day.

After processing: 
 ['Hey', 'We', 'using', 'bot', 'And', 'also', 'filtering', 'recommendations', 'select', 'topics', 'worth', 'attention', '~5', 'people', 'duty', 'every', 'day']


In [44]:
target_labels

array(['Auroracoin', 'BitConnect (inactive)', 'Bitcoin Cash', 'Bitcoin',
       'Dash', 'Dogecoin', 'EOS.IO', 'Ethereum Classic',
       'Ether (also known as Ethereum)', 'Gridcoin', 'Litecoin',
       'Coinye (inactive)', 'Mazacoin', 'Nano', 'Neo', 'Namecoin', 'NXT',
       'PotCoin', 'Peercoin', 'Titcoin', 'USD Coin (stablecoin)',
       'Tether', 'Vertcoin', 'NEM', 'Stellar', 'Monero', 'Primecoin',
       'Ripple', 'Verge', 'Zcash', 'Bitcoin Improvement Proposal',
       'Automatic Teller Machine for Bitcoin',
       'Enterprise Ethereum Alliance', 'Ethereum Improvement Proposal',
       'Fundamental Analysis', 'Moving Average Convergence Divergence',
       'Proof of Burn', 'Proof of Developer',
       'Technical Analysis or Trend Analysis',
       'Coordinated Universal Time', 'Year to Date', 'Address',
       'Application Programming Interface', 'Brute Force Attack',
       'Bitcoin address format (also known as bc1 addresses)',
       'Developers Conference', 'Multi-Signature',


In [45]:
# remove abbreviations from labels
final_labels = []
for labels in target_labels:
    for x in custom_mapping.keys():
        labels = re.sub(custom_mapping[x], x, labels)
    final_labels.append(labels)
final_labels

['Auroracoin',
 'BitConnect (inactive)',
 'Bitcoin Cash',
 'Bitcoin',
 'Dash',
 'Dogecoin',
 'EOS.IO.IO',
 'Ethereum Classic',
 'Ether (also known as Ethereum)',
 'Gridcoin',
 'Litecoin',
 'Coinye (inactive)',
 'Mazacoin',
 'Nano',
 'Neo',
 'Namecoin',
 'NXT',
 'PotCoin',
 'Peercoin',
 'Titcoin',
 'USD Coin (stablecoin)',
 'Tether',
 'Vertcoin',
 'NEM',
 'Stellar',
 'Monero',
 'Primecoin',
 'Ripple',
 'Verge',
 'Zcash',
 'Bitcoin Improvement Proposal',
 'Automatic Teller Machine for Bitcoin',
 'Enterprise Ethereum Alliance',
 'Ethereum Improvement Proposal',
 'Fundamental Analysis',
 'Moving Average Convergence Divergence',
 'Proof of Burn',
 'Proof of Developer',
 'Technical Analysis or Trend Analysis',
 'Coordinated Universal Time',
 'Year to Date',
 'Address',
 'Application Programming Interface',
 'Brute Force Attack',
 'Bitcoin address format (also known as bc1 addresses)',
 'Developers Conference',
 'Multi-Signature',
 'Secure Hash Acronym (256-bit)',
 'Worldwide Developers Confe

In [46]:
# tokenize conversations and pos tag
tokens = process_sentence(example)
marked_tags = nltk.pos_tag(tokens) 
df = pd.DataFrame(marked_tags).T
df.index = ["Tokens", "POS Tag"]
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Tokens,Hey,We,using,bot,And,also,filtering,recommendations,select,topics,worth,attention,~5,people,duty,every,day
POS Tag,NNP,PRP,VBG,NN,CC,RB,VBG,NNS,VBP,NNS,JJ,NN,JJ,NNS,VBP,DT,NN


In [17]:
def cosine_distance(x, y):
    return 1 - np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [30]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_lg')
# process a sentence using the model
doc = nlp("This is some text that I am processing with Spacy")

In [21]:
# doc[3].vector #for word index i embeddings
# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector.shape

(300,)

In [74]:
emb_final_labels = ''

for label in final_labels:
    emb_final_labels += label + ' '

emb_final_labels = nlp(emb_final_labels).vector
emb_final_labels.shape

(300,)

# Extracting Phrases

In [95]:
from sklearn.metrics.pairwise import cosine_similarity

all_conversations_embedding = []

for example in all_conversations[:1000]:
    tokens = process_sentence(example)
    marked_tags = nltk.pos_tag(tokens) 

    back_to_sentence = ''
    for word, tag in marked_tags:
        if tag == 'NN' or tag == 'NNP' or tag == 'NNS' or tag == 'JJ':
            back_to_sentence += word + ' '
            
    sentence_embedding = nlp(back_to_sentence).vector
    all_conversations_embedding.append(sentence_embedding)
    # calculate cosine distance between conversation and mean of label text
    score = cosine_similarity([sentence_embedding], [emb_final_labels])
    if score > 0.50:
        print(score, back_to_sentence)

[[0.586259]] non-official one 
[[0.5016016]] know small traders governance forums accurate mental model 
[[0.5360624]] account single NFT available buy target Ether Ethereum 
[[0.6373631]] reference 
[[0.54820114]] additional yield 
[[0.5252242]] ~8k Ether Ethereum withdrawal noticeable wallets 
[[0.57355946]] low-quality bLuE-cRyPtO-sHaRk NFT project 
[[0.5388564]] Instructions LooksRare fork NFT project 
[[0.56355906]] continuation cex world pre chinese exchanges billions volume coinbase mm 
[[0.5633209]] Chinese exchanges billions volumes 
[[0.6462274]] Total Ether Ethereum prob 
[[0.5343522]] ~500 unique users week ~4-6k Ether Ethereum withdrawals month 1-1.5 week withdrawals 4-5 Ether Ethereum average week pointers 
[[0.52600956]] DAO Daniele Asserts Order 
[[0.5719805]] DAO Daniele Operation 
[[0.62113124]] CDO Centralized Dependent Organisation Did meant vote/ultimo leader style create proposal Nothing change gov process bluff 
[[0.52089965]] Sounds complex 
[[0.5557653]] Hardwa

In [ ]:
# !gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM

# import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)  

# model.get_vector("Auroracoin")

# "word 'Auroracoin' not in vocabulary"